In [2]:
from os import environ
from dotenv import load_dotenv
import numpy as np
import pandas as pd
from riotwatcher import LolWatcher, ApiError

load_dotenv("../")

RIOT_API = environ["RIOT_API"]
lol_watcher = LolWatcher(RIOT_API)
my_region = "na1"
NA = "na1"

In [3]:
yogurtsauce = pd.DataFrame.from_dict(
    lol_watcher.summoner.by_name(NA, "yogurtsauce"), orient="index"
).T

In [4]:
ranked = lol_watcher.league.by_summoner(NA, yogurtsauce.id[0])
versions = lol_watcher.data_dragon.versions_for_region(NA)
champions_version = versions["n"]["champion"]
summoner_spells_version = versions["n"]["summoner"]
items_version = versions["n"]["item"]
current_champ_list = lol_watcher.data_dragon.champions(champions_version)
my_matches = lol_watcher.match.matchlist_by_puuid(NA, yogurtsauce.puuid[0])
current_items = lol_watcher.data_dragon.items(items_version)
mastery_version = versions["n"]["mastery"]

In [5]:
champions = (
    pd.DataFrame.from_dict(current_champ_list)
    .reset_index()
    .rename(columns={"index": "champion"})
)

champions = pd.concat([champions, pd.json_normalize(champions.data)], axis=1).drop(
    columns=["type", "format", "version", "data", "name", "id"]
)

champions.insert(0, "key", champions.pop("key"))
champions[["primary_role", "secondary_role"]] = pd.DataFrame(
    champions.tags.tolist(), index=champions.index
)
champions = champions.drop(columns="tags")
champions.primary_role = champions.primary_role.astype("category")
champions.secondary_role = champions.secondary_role.astype("category")
champions.partype = champions.partype.astype("category")
champions.head()

,key,champion,title,blurb,partype,info.attack,info.defense,info.magic,info.difficulty,image.full,...,stats.mpregen,stats.mpregenperlevel,stats.crit,stats.critperlevel,stats.attackdamage,stats.attackdamageperlevel,stats.attackspeedperlevel,stats.attackspeed,primary_role,secondary_role
0,266,Aatrox,the Darkin Blade,Once honored defenders of Shurima against the ...,Blood Well,8,4,3,4,Aatrox.png,...,0.0,0.0,0,0,60,5.00,2.500,0.651,Fighter,Tank
1,103,Ahri,the Nine-Tailed Fox,Innately connected to the magic of the spirit ...,Mana,3,4,8,5,Ahri.png,...,8.0,0.8,0,0,53,3.00,2.000,0.668,Mage,Assassin
2,84,Akali,the Rogue Assassin,Abandoning the Kinkou Order and her title of t...,Energy,5,3,8,7,Akali.png,...,50.0,0.0,0,0,62,3.30,3.200,0.625,Assassin,NaN
3,166,Akshan,the Rogue Sentinel,"Raising an eyebrow in the face of danger, Aksh...",Mana,0,0,0,0,Akshan.png,...,8.2,0.7,0,0,52,3.50,4.000,0.638,Marksman,Assassin
4,12,Alistar,the Minotaur,Always a mighty warrior with a fearsome reputa...,Mana,6,9,5,7,Alistar.png,...,8.5,0.8,0,0,62,3.75,2.125,0.625,Tank,Support


In [12]:
champions.columns

Index(['key', 'champion', 'title', 'blurb', 'partype', 'info.attack',
       'info.defense', 'info.magic', 'info.difficulty', 'image.full',
       'image.sprite', 'image.group', 'image.x', 'image.y', 'image.w',
       'image.h', 'stats.hp', 'stats.hpperlevel', 'stats.mp',
       'stats.mpperlevel', 'stats.movespeed', 'stats.armor',
       'stats.armorperlevel', 'stats.spellblock', 'stats.spellblockperlevel',
       'stats.attackrange', 'stats.hpregen', 'stats.hpregenperlevel',
       'stats.mpregen', 'stats.mpregenperlevel', 'stats.crit',
       'stats.critperlevel', 'stats.attackdamage',
       'stats.attackdamageperlevel', 'stats.attackspeedperlevel',
       'stats.attackspeed', 'primary_role', 'secondary_role'],
      dtype='object')

In [6]:
class Summoner:
    def __init__(self, region, summoner_name):
        self.region = region
        self.name = summoner_name
        self.data = pd.DataFrame.from_dict(
            lol_watcher.summoner.by_name(self.region, self.name), orient="index"
        ).T
        self.id = self.data.id[0]
        self.account_id = self.data.accountId[0]
        self.puuid = self.data.puuid[0]
        self.profile_icon_id = self.data.profileIconId[0]
        self.revision_date = self.data.revisionDate[0]
        self.level = self.data.summonerLevel[0]
        self.matches_ = lol_watcher.match.matchlist_by_puuid(self.region, self.puuid)


yogurtsauce = Summoner("NA1", "yogurtsauce")

In [7]:
def match_history(username, region, n_games=5):
    i = 0
    j = 0
    user = Summoner(region, username)
    matches = pd.DataFrame()

    while j < n_games:
        try:
            last_match = user.matches_[j]
            match_details = lol_watcher.match.by_id(region, last_match)
            match_players = match_details["info"]["participants"]
            details = []

            for player in match_players:
                dict = {}
                dict["ign"] = player["summonerName"]
                dict["level"] = player["summonerLevel"]
                dict["champion_id"] = player["championId"]
                dict["champion"] = player["championName"]
                dict[
                    "mastery_points"
                ] = lol_watcher.champion_mastery.by_summoner_by_champion(
                    region, player["summonerId"], player["championId"]
                )[
                    "championPoints"
                ]
                dict["team"] = player["teamId"]
                dict["win"] = player["win"]

                details.append(dict)

            details = pd.DataFrame.from_dict(details)
            details = details.merge(
                champions[
                    [
                        "champion",
                        "primary_role",
                        "secondary_role",
                        "info.attack",
                        "info.defense",
                        "info.magic",
                        "info.difficulty",
                        "partype"
                    ]
                ]
            )
            
            details.team = details.team.astype("category")

            matches = pd.concat([matches, details], ignore_index=True)

            i += 1
            j += 1
        except:
            j += 1

    return matches


test = match_history("yogurtsauce", "na1", 1)

In [17]:
test

,ign,level,champion_id,champion,mastery_points,team,win,primary_role,secondary_role,info.attack,info.defense,info.magic,info.difficulty,partype
0,my3dwarfrogs,465,80,Pantheon,30788,100,False,Fighter,Assassin,9,4,3,4,Mana
1,Syxj,232,202,Jhin,6676,100,False,Marksman,Mage,10,2,6,6,Mana
2,yogurtsauce,514,1,Annie,12895,100,False,Mage,NaN,2,3,10,6,Mana
3,Gogeta UI,279,51,Caitlyn,52011,100,False,Marksman,NaN,8,2,2,6,Mana
4,Temporal Logics,357,31,Chogath,19551,100,False,Tank,Mage,3,7,7,5,Mana
5,itsdango,402,517,Sylas,49330,200,True,Mage,Assassin,3,4,8,5,Mana
6,botfrag,367,136,AurelionSol,11574,200,True,Mage,NaN,2,3,8,7,Mana
7,FATHER GYOZA,496,164,Camille,17097,200,True,Fighter,Tank,8,6,3,4,Mana
8,HuyAnh Maii,681,876,Lillia,16379,200,True,Fighter,Mage,0,2,10,8,Mana
9,A Real Struggle,567,127,Lissandra,68822,200,True,Mage,NaN,2,5,8,6,Mana


In [9]:
wins = test.where(test.win == True)
wins = wins[~wins["ign"].isnull()]

losses = test.where(test.win == False)
losses = losses[~losses["ign"].isnull()]

In [19]:
wins["champion1"] = wins["champion"].str.split().str[0]
wins["champion2"] = wins["champion"].str.split().str[1]
wins["champion3"] = wins["champion"].str.split().str[3]
wins["champion4"] = wins["champion"].str.split().str[4]
wins["champion5"] = wins["champion"].str.split().str[5]

In [ ]:
for x in 

In [20]:
wins

,ign,level,champion_id,champion,mastery_points,team,win,primary_role,secondary_role,info.attack,info.defense,info.magic,info.difficulty,partype,champion1,champion2,champion3,champion4,champion5
5,itsdango,402.0,517.0,Sylas,49330.0,200,True,Mage,Assassin,3.0,4.0,8.0,5.0,Mana,Sylas,NaN,NaN,NaN,NaN
6,botfrag,367.0,136.0,AurelionSol,11574.0,200,True,Mage,NaN,2.0,3.0,8.0,7.0,Mana,AurelionSol,NaN,NaN,NaN,NaN
7,FATHER GYOZA,496.0,164.0,Camille,17097.0,200,True,Fighter,Tank,8.0,6.0,3.0,4.0,Mana,Camille,NaN,NaN,NaN,NaN
8,HuyAnh Maii,681.0,876.0,Lillia,16379.0,200,True,Fighter,Mage,0.0,2.0,10.0,8.0,Mana,Lillia,NaN,NaN,NaN,NaN
9,A Real Struggle,567.0,127.0,Lissandra,68822.0,200,True,Mage,NaN,2.0,5.0,8.0,6.0,Mana,Lissandra,NaN,NaN,NaN,NaN


In [10]:
losses

,ign,level,champion_id,champion,mastery_points,team,win,primary_role,secondary_role,info.attack,info.defense,info.magic,info.difficulty,partype
0,my3dwarfrogs,465.0,80.0,Pantheon,30788.0,100,False,Fighter,Assassin,9.0,4.0,3.0,4.0,Mana
1,Syxj,232.0,202.0,Jhin,6676.0,100,False,Marksman,Mage,10.0,2.0,6.0,6.0,Mana
2,yogurtsauce,514.0,1.0,Annie,12895.0,100,False,Mage,NaN,2.0,3.0,10.0,6.0,Mana
3,Gogeta UI,279.0,51.0,Caitlyn,52011.0,100,False,Marksman,NaN,8.0,2.0,2.0,6.0,Mana
4,Temporal Logics,357.0,31.0,Chogath,19551.0,100,False,Tank,Mage,3.0,7.0,7.0,5.0,Mana


In [11]:
wins

,ign,level,champion_id,champion,mastery_points,team,win,primary_role,secondary_role,info.attack,info.defense,info.magic,info.difficulty,partype
5,itsdango,402.0,517.0,Sylas,49330.0,200,True,Mage,Assassin,3.0,4.0,8.0,5.0,Mana
6,botfrag,367.0,136.0,AurelionSol,11574.0,200,True,Mage,NaN,2.0,3.0,8.0,7.0,Mana
7,FATHER GYOZA,496.0,164.0,Camille,17097.0,200,True,Fighter,Tank,8.0,6.0,3.0,4.0,Mana
8,HuyAnh Maii,681.0,876.0,Lillia,16379.0,200,True,Fighter,Mage,0.0,2.0,10.0,8.0,Mana
9,A Real Struggle,567.0,127.0,Lissandra,68822.0,200,True,Mage,NaN,2.0,5.0,8.0,6.0,Mana


In [226]:
test.dtypes

ign                  object
level                 int64
champion_id           int64
champion             object
mastery_points        int64
team               category
win                    bool
primary_role       category
secondary_role     category
info.attack           int64
info.defense          int64
info.magic            int64
info.difficulty       int64
partype            category
dtype: object